# Import dependencies

In [63]:
import fnmatch
import os
import keypoint_moseq as kpms
from tkinter import Tk
from tkinter.filedialog import askdirectory
import matplotlib
matplotlib.use('Agg')

# Find Files

In [64]:
# Find file
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files: 
            if fnmatch.fnmatch(name,pattern): 
                result.append(os.path.join(root,name))
    # print(len(result))
    if len(result)==1:
        result = result[0]
    return result

In [65]:
path = r'D:\Barrier_testing_day1_videos'
project_dir = path
config = lambda: kpms.load_config(project_dir)

dlc_config = find('config.yaml',path)

In [66]:
path

'D:\\Barrier_testing_day1_videos'

In [67]:
# print(dlc_config)
kpms.setup_project(project_dir, deeplabcut_config=dlc_config, overwrite=False)
video_paths = find("*.mp4v", path)
kpms.update_config(
    project_dir,
    video_paths=video_paths,
    anterior_bodyparts=['Nose'],
    posterior_bodyparts=['Tail_Base'],
    use_bodyparts=['Back_Left_Leg','Back_Right_Leg','Left_Ear','Middle_Neck','Middle_Spine','Nose','Right_Ear','Tail_Base','Tail_End','Tail_Mid'])

The directory `D:\Barrier_testing_day1_videos` already exists. Use
`overwrite=True` or pick a different name


In [68]:
# load data (e.g. from DeepLabCut)
keypoint_data_path = find("*cm_*Effort*filtered.h5", path) # can be a file, a directory, or a list of files
for file in keypoint_data_path:
    print(file)
coordinates, confidences, bodyparts = kpms.load_keypoints(keypoint_data_path, 'deeplabcut')

D:\Barrier_testing_day1_videos\031524_Barrier_Testing_10cm_Day_1_WT027MControl_StartArm_0DLC_resnet50_Effort_Related_Choice_2Jul30shuffle1_100000_filtered.h5
D:\Barrier_testing_day1_videos\031524_Barrier_Testing_10cm_Day_1_WT027MControl_StartArm_10DLC_resnet50_Effort_Related_Choice_2Jul30shuffle1_100000_filtered.h5
D:\Barrier_testing_day1_videos\031524_Barrier_Testing_10cm_Day_1_WT027MControl_StartArm_1DLC_resnet50_Effort_Related_Choice_2Jul30shuffle1_100000_filtered.h5
D:\Barrier_testing_day1_videos\031524_Barrier_Testing_10cm_Day_1_WT027MControl_StartArm_2DLC_resnet50_Effort_Related_Choice_2Jul30shuffle1_100000_filtered.h5
D:\Barrier_testing_day1_videos\031524_Barrier_Testing_10cm_Day_1_WT027MControl_StartArm_3DLC_resnet50_Effort_Related_Choice_2Jul30shuffle1_100000_filtered.h5
D:\Barrier_testing_day1_videos\031524_Barrier_Testing_10cm_Day_1_WT027MControl_StartArm_4DLC_resnet50_Effort_Related_Choice_2Jul30shuffle1_100000_filtered.h5
D:\Barrier_testing_day1_videos\031524_Barrier_Testi

Loading keypoints: 100%|██████████████| 878/878 [00:10<00:00, 84.07it/s]


# Modeling

In [69]:
# format data for modeling
data, metadata = kpms.format_data(coordinates, confidences, **config())

pca = kpms.fit_pca(**data, **config())
kpms.save_pca(pca, project_dir)

kpms.print_dims_to_explain_variance(pca, 0.9)
kpms.plot_scree(pca, project_dir=project_dir)

>=90.0% of variance exlained by 7 components.


c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:112: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



<Figure size 300x200 with 1 Axes>

In [70]:
# initialize the model
model = kpms.init_model(data, pca=pca, **config())

num_ar_iters = 50
num_ar_iters = 50
model, model_name = kpms.fit_model(
    model, data, metadata, project_dir,
    ar_only=True, num_iters=num_ar_iters)

from jax_moseq.utils import set_mixed_map_iters
set_mixed_map_iters(8)


Outputs will be saved to
D:\Barrier_testing_day1_videos\2025_06_23-09_34_57


 49%|█████████████████▏                 | 25/51 [00:15<00:12,  2.03it/s]c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:655: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

 98%|██████████████████████████████████▎| 50/51 [00:28<00:00,  2.09it/s]c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:655: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

100%|███████████████████████████████████| 51/51 [00:29<00:00,  1.73it/s]


In [71]:
# load model checkpoint
model, data, metadata, current_iter = kpms.load_checkpoint(
    project_dir, model_name, iteration=num_ar_iters)

In [72]:
# modify kappa to maintain the desired syllable time-scale
model = kpms.update_hypparams(model, kappa=4e2)

In [73]:
# run fitting for an additional 200 iters
model = kpms.fit_model(
    model, data, metadata, project_dir, model_name, ar_only=False, 
    start_iter=current_iter, num_iters=current_iter+100,parallel_message_passing=False)[0]
kpms.reindex_syllables_in_checkpoint(project_dir, model_name)

Outputs will be saved to
D:\Barrier_testing_day1_videos\2025_06_23-09_34_57


 25%|████████▍                         | 25/101 [05:13<15:15, 12.05s/it]c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:655: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

 50%|████████████████▊                 | 50/101 [10:09<10:03, 11.83s/it]c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:655: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

 74%|█████████████████████████▏        | 75/101 [15:10<05:14, 12.10s/it]c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:655: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

 99%|████████████████████████████████▋| 100/101 [20:07<00:11, 11.52s/it]c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:655: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

Reindexing: 

array([1, 2, 4, 7, 5, 6, 3, 0], dtype=int64)

In [74]:
# load the most recent model checkpoint
model, data, metadata, current_iter = kpms.load_checkpoint(project_dir, model_name)

In [88]:
# extract results
results = kpms.extract_results(model, metadata, project_dir, model_name)

results = kpms.load_results(project_dir, model_name)
kpms.generate_trajectory_plots(coordinates, results, project_dir, model_name, sampling_options={"mode": "density", "n_neighbors": 20}, min_frequency=0.005, **config())
kpms.save_results_as_csv(results, project_dir, model_name)

Saved results to
D:\Barrier_testing_day1_videos\2024_10_15-16_45_52\results.h5
Saving trajectory plots to D:\Barrier_testing_day1_videos\2024_10_15-16_45_52\trajectory_plots


Generating trajectory plots: 100%|██████| 15/15 [00:04<00:00,  3.35it/s]
c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:1810: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

Saving to csv: 100%|█████████████████| 878/878 [00:05<00:00, 164.34it/s]


In [ ]:
# import matplotlib.pyplot as plt

# plt.rcParams.update({'font.size': 12})  # <-- Change global font size

# fig, ax = plt.subplots(figsize=(8, 6))  # Set the figure size here

# kpms.generate_trajectory_plots(

#     coordinates,
#     results,
#     project_dir,
#     model_name,
#     sampling_options={"mode": "density", "n_neighbors": 20},
#     **config()
# )

# plt.tight_layout()
# plt.savefig("syllable_trajectories.pdf", format="pdf")
# plt.show()


Saving trajectory plots to D:\Barrier_testing_day1_videos\2024_10_15-16_45_52\trajectory_plots


Generating trajectory plots: 100%|██████| 15/15 [00:03<00:00,  4.00it/s]
c:\Users\Jillian.Sucher\anaconda3\envs\keypoint_moseq\lib\site-packages\keypoint_moseq\viz.py:1810: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\Jillian.Sucher\AppData\Local\Temp\ipykernel_11952\1597921721.py:19: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



In [77]:
# Assume the figure is created by kpms.generate_trajectory_plots
ax = plt.gca()

# Move x-axis label downward using labelpad or set_position
ax.set_xlabel(ax.get_xlabel(), labelpad=15)  # Increase labelpad to move down
# OR use set_position for full control (x, y)
ax.xaxis.set_label_coords(0.5, -0.1)  # 0.5 = center; -0.1 moves it down

# Similarly for y-axis (move it horizontally if needed)
ax.set_ylabel(ax.get_ylabel(), labelpad=15)
# ax.yaxis.set_label_coords(-0.1, 0.5)  # Uncomment if needed

plt.tight_layout()
plt.savefig("syllable_trajectories.pdf", format="pdf")
plt.show()



C:\Users\Jillian.Sucher\AppData\Local\Temp\ipykernel_11952\2992507702.py:15: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



# Figs

In [89]:
kpms.plot_similarity_dendrogram(coordinates, results, project_dir, model_name, figsize=(10, 8), **config())

Saving dendrogram plot to D:\Barrier_testing_day1_videos\2024_10_15-16_45_52\similarity_dendrogram


In [79]:
import matplotlib.pyplot as plt
import seaborn as sns

In [80]:
# Assuming this is your dendrogram plotting line:
#fig, ax = kpms.plot_similarity_dendrogram(coordinates, results, project_dir, model_name, figsize=(10, 8), **config())

#ax = plt.gca()  # get current axis

kpms.plot_similarity_dendrogram(coordinates, results, project_dir, model_name, figsize=(6, 4), **config())
ax = plt.gca()  # get current axis

# Remove the title
ax.set_title('')

# # Remove "syllable" from y-tick labels
# yticklabels = [label.get_text().replace('syllable ', '') for label in ax.get_yticklabels()]
# ax.set_yticklabels(yticklabels, fontsize=20)  # Adjust font size as needed

import matplotlib.ticker as ticker

# Draw figure first so tick labels are populated
plt.draw()

# Get current y-tick labels and positions
ytick_labels = ax.get_yticklabels()
ytick_positions = ax.get_yticks()

# Extract plain text, remove "syllable "
cleaned_labels = [label.get_text().replace('syllable ', '') for label in ytick_labels]

# Update ticks and apply FixedFormatter
ax.set_yticks(ytick_positions)
ax.yaxis.set_major_formatter(ticker.FixedFormatter(cleaned_labels))

# Update font size for labels
for label in ax.get_yticklabels():
    label.set_fontsize(20)


# Remove top and right spines
sns.despine(ax=ax, top=True, right=True)

# Set all dendrogram lines to black
for collection in ax.collections:
    collection.set_color('black')
    collection.set_linewidth(1.5)

# # Set all line colors in the dendrogram to black
# for line in ax.get_lines():
#     line.set_color('black')
#     line.set_linewidth(1.5)  # Optional: adjust thickness

# Optional: remove y-axis label if present
ax.set_ylabel('')

plt.tight_layout()
plt.savefig("dendrogram.pdf", format="pdf")
plt.show()


Saving dendrogram plot to D:\Barrier_testing_day1_videos\2025_06_23-09_34_57\similarity_dendrogram


C:\Users\Jillian.Sucher\AppData\Local\Temp\ipykernel_11952\3224182124.py:55: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



## loading code is in apply to new data section of: https://keypoint-moseq.readthedocs.io/en/latest/modeling.html

In [81]:
import keypoint_moseq as kpms

path = r'D:\Barrier_testing_day1_videos'
project_dir = path
model_name='2024_10_15-16_45_52'

# load the most recent model checkpoint and pca object
model = kpms.load_checkpoint(project_dir, model_name)[0]

# load new data (e.g. from deeplabcut)
#new_data = 'path/to/new/data/' # can be a file, a directory, or a list of files
keypoint_data_path = find("*cm_*Effort*filtered.h5", path) # can be a file, a directory, or a list of files
coordinates, confidences, bodyparts = kpms.load_keypoints(keypoint_data_path, 'deeplabcut')
data, metadata = kpms.format_data(coordinates, confidences, **config())

# apply saved model to new data
#results = kpms.apply_model(model, data, metadata, project_dir, model_name, **config())


#save_results_as_csv
# kpms.save_results_as_csv(results, project_dir, model_name)

Loading keypoints: 100%|█████████████| 878/878 [00:04<00:00, 179.77it/s]


In [82]:
# kpms.generate_grid_movies(results, project_dir, model_name, coordinates=coordinates, **config());

In [83]:
# # Assuming you have a function config() that loads your configuration
# config_data = config()

# # Print the entire configuration data
# print("Configuration data:", config_data)

# # Specifically check video_dir and video_paths
# print("Video Directory:", config_data.get('video_dir'))
# print("Video Paths:", config_data.get('video_paths'))

In [84]:
results = kpms.load_results(project_dir, model_name)
# kpms.generate_grid_movies(results, project_dir, model_name, coordinates=coordinates, **config());

In [85]:
# video_dir = r'c:\Users\Jillian.Sucher\Documents\Stress_microstructure_testing_day_1'
# video_paths = find('*cm_*Effort*.avi',path)
# keys = [name[:-4] for name in video_paths]
# video_paths = kpms.find_matching_videos(keys,path,as_dict=True)